In [1]:
import math
import pandas as pd
import Model_LSTM as kr
import Embedding as yy
import numpy as np
from keras.preprocessing.text import Tokenizer # https://keras-cn.readthedocs.io/en/latest/preprocessing/text/
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\S5\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
if __name__=='__main__':

    # adjustable parameter
    offset=2 # l1=l3=l2+offset. namely l1 refers to get length(truncated length), l2 refers to average length, l3 refers to train length
    rm_symbols='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    embedding_dim=50


In [4]:
    # read dataset and dictionary
    data_train=pd.read_csv('../dataset/Train.csv')
    
    X_train=data_train['TEXT'].values
    Y_train=data_train['Label'].values
    

    #Y_train[Y_train == 5] = 1
    Y_train[Y_train == 6] = 1
    Y_train[Y_train == 11] = 1
    Y_train[Y_train == 12] = 1
    Y_train[Y_train == 13] = 1
 


    Y_train[Y_train == 7] = 2
    Y_train[Y_train == 15] = 2
    Y_train[Y_train == 8] = 2
    Y_train[Y_train == 14] = 2
    Y_train[Y_train == 18] = 2
    Y_train[Y_train == 9] = 2
    

    
    #Y_train[Y_train == 0] = 4
    Y_train[Y_train == 16] = 4
    Y_train[Y_train == 19] = 4

    
    Y_train[Y_train == 17] = 10
    
    
    
    '''
    以下有待商榷
    
    Y_train[Y_train == 2] = 1
    Y_train[Y_train == 4] = 1
    Y_train[Y_train == 10] = 1
    '''

    


    #Y_123 = Y_train[:]
    

    Y_train=to_categorical(Y_train,20)
    X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)
    
    #data_test=pd.read_csv('../dataset/Test.csv')
   # X_test=data_test['TEXT'].values
    
    f='../dataset/glove.6B.50d.txt'
    
    emoji_map = pd.read_csv('../dataset/Mapping.csv')

In [5]:
print(Y_train[1,:])

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [6]:
    # remove special symbols and stopwords from train set
    X_rm=yy.corpus_pre(X_train)

    # segmentation
    tokenizer = Tokenizer(filters=rm_symbols, split=" ", lower=True) # filters：filter symbols that need to be removed lower：convert to lowercase
    tokenizer.fit_on_texts(X_rm) # Tokenizer read train set free of special symbols. Results are stored in tokenize handle.
    
    # vectorize. fill in and truncation
    l2 = math.ceil(sum([len(s.split(" ")) for s in X_rm])/len(X_rm)) # l2:average length
    l1 = l2+offset #get length (truncated length)
    X_pd,tokenizer = kr.toknz(X_rm, l1,tokenizer)

In [7]:
    #Dict that allocate an id(integer) to every word
    ind_dict=tokenizer.word_index

    #Dict that allocate an word vector to every word
    lookup_dict=yy.dict_pre(f)

    #Generate weightMatrix according to dictionary
    W=yy.lookup(ind_dict,lookup_dict,embedding_dim)


In [16]:
    # train
    model=kr.model_training(len(ind_dict)+1, W, l2+offset, X_pd, Y_train, embed_dim=embedding_dim, epochs=5)
    print(model.predict_classes(X_pd[1:13])) #test on some sentences in the train data set

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 13, 50)            2257600   
_________________________________________________________________
lstm_3 (LSTM)                (None, 13, 128)           91648     
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 20)                2580      
Total params: 2,483,412
Trainable params: 2,483,412
Non-trainable params: 0
_________________________________________________________________


C:\Users\S5\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 47600 samples, validate on 8400 samples
Epoch 1/4
47600/47600 [==============================] - 28s 595us/step - loss: 1.4494 - accuracy: 0.5198 - val_loss: 1.3339 - val_accuracy: 0.5287
Epoch 2/4
47600/47600 [==============================] - 28s 579us/step - loss: 1.2901 - accuracy: 0.5445 - val_loss: 1.2644 - val_accuracy: 0.5467
Epoch 3/4
47600/47600 [==============================] - 27s 568us/step - loss: 1.2108 - accuracy: 0.5701 - val_loss: 1.2282 - val_accuracy: 0.5658
Epoch 4/4
56000/56000 [==============================] - 9s 155us/step
[2 2 2 2 2 2 2 1 5 3 3 1]


## Predict on test set

In [17]:
    # Prediction on test set
    X_test_rm = yy.corpus_pre(X_test)
    X_test_pd,_ = kr.toknz(X_test_rm, l1,tokenizer)
    label_test = model.predict_classes(X_test_pd)
    #for i in range(500, 521, 1):
        #print(emoji_map['emoticons'][label_test[i]])
        #print(X_test[i])
    

In [18]:
label_test.shape

(14000,)

In [19]:
label_test=to_categorical(label_test,20)

In [20]:
accuracy = 0
inaccuracy = []
for i in range(len(Y_test)):
    if (label_test[i,:] == Y_test[i,:]).all():
        accuracy += 1

accuracy = accuracy/len(Y_test)
        
        

In [21]:
accuracy

0.571

## Predict on user input

In [14]:
    
    #user_str = input("input your sentence:")   
    #user_str = "I love you"
    #X_user = np.array([str(user_str)])
   # print(X_user[0])

In [15]:
    #X_user_rm = yy.corpus_pre(X_user)
    #X_user_pd,_ = kr.toknz(X_user_rm, l1,tokenizer)
    #label_user = model.predict_classes(X_user_pd)
    #print(emoji_map['emoticons'][label_user[0]])
    #print(X_user[0]) 